# IBM Data Science Professional Certificate Capstone

This notebook is for the neighborhood analyzation project for the data science capstone course on Coursera.

# Part 1 - Identifying our districts

In [1]:
# Our needed imports.
#!conda install -c conda-forge folium --yes
#!conda install -c conda-forge geopy --yes
import folium
import ibm_boto3
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
import pandas as pd
import requests
import types
from botocore.client import Config
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from IPython.display import Image 
from sklearn.cluster import KMeans

In [2]:
# Create our corners of Gainesville.
gainesville_north = 29.711381
gainesville_south = 29.596737
gainesville_west = -82.453961
gainesville_east = -82.262119

In [3]:
# Define how many rows and columns we want to create for districts.
DISTRICT_ROWS = 11
DISTRICT_COLUMNS = 16
GAINESVILLE_LATITUDE = 29.662737
GAINESVILLE_LONGITUDE = -82.370212

In [4]:
# Calculate how big each segment is.
lat_diff = gainesville_north - gainesville_south
long_diff = gainesville_west - gainesville_east
lat_segment = lat_diff / (DISTRICT_ROWS)
long_segment = long_diff / (DISTRICT_COLUMNS)

In [5]:
# Generate the center for all segments.
gainesville_districts = pd.DataFrame(columns=['District', 'Lat', 'Long'])
north_boundary = gainesville_north
for row in range(DISTRICT_ROWS):
    south_boundary = north_boundary - lat_segment
    row_center = (north_boundary + south_boundary) / 2
    west_boundary = gainesville_west
    for column in range(DISTRICT_COLUMNS):
        east_boundary = west_boundary - long_segment
        column_center = (east_boundary + west_boundary) / 2
        west_boundary = east_boundary
        gainesville_districts = gainesville_districts.append(pd.Series(['{}-{}'.format(row, column), row_center, column_center], index=gainesville_districts.columns), ignore_index=True)
    north_boundary = south_boundary
gainesville_districts.head()

,District,Lat,Long
0,0-0,29.70617,-82.447966
1,0-1,29.70617,-82.435976
2,0-2,29.70617,-82.423986
3,0-3,29.70617,-82.411996
4,0-4,29.70617,-82.400005


In [7]:
# Create map of Gainesville to see our districts.
general_map = folium.Map(location=[GAINESVILLE_LATITUDE, GAINESVILLE_LONGITUDE], zoom_start=12)

# Add markers to the map for each districts.
for index, row in gainesville_districts.iterrows():
    folium.CircleMarker(
        [row['Lat'], row['Long']],
        radius=17,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(general_map)  
    
general_map

# Part 2 - Analyzing the districts

## Get the businesses in each district from FourSquare

In [8]:
# The code was removed by Watson Studio for sharing.

In [9]:
# Sets up our categories
food_category = '4d4b7105d754a06374d81259'

In [21]:
# Prepares our venue DataFrame.
neighborhood_venues = pd.DataFrame(columns=[
                            'District',
                            'District Latitude',
                            'District Longitude', 
                            'Venue', 
                            'Venue Latitude', 
                            'Venue Longitude', 
                            'Venue Category'])

In [11]:
# Function for getting all venues in an area
def get_venues(lat, long, radius, limit):
    # create the API request URL.
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&categoryId={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION,
        food_category,
        lat, 
        long, 
        radius, 
        limit)
    
    # Load our results.
    r = requests.get(url)
    results = r.json()
    
    # Get the venues.
    try:
        venues = results["response"]['groups'][0]['items']
        return venues
    except KeyError:
        print('Trouble finding venues for {}. Returned response was:'.format(row['District']), results["response"])
    return []
        

In [22]:
district_count = len(gainesville_districts)
for index, row in gainesville_districts.iterrows():
    if index % 10 == 0:
        print('District {} of {}...'.format(index, district_count))
    venues = get_venues(row['Lat'], row['Long'], 500, 100)
    # Add each venue to our DataFrame.
    for venue in venues:
        neighborhood_venues = neighborhood_venues.append(pd.Series([
            row['District'],
            row['Lat'],
            row['Long'],
            venue['venue']['name'],
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']], index=neighborhood_venues.columns), ignore_index=True)

District 0 of 176...
District 10 of 176...
District 20 of 176...
District 30 of 176...
District 40 of 176...
District 50 of 176...
District 60 of 176...
District 70 of 176...
District 80 of 176...
District 90 of 176...
District 100 of 176...
District 110 of 176...
District 120 of 176...
District 130 of 176...
District 140 of 176...
District 150 of 176...
District 160 of 176...
District 170 of 176...


In [23]:
# Quick preview of our venues.
print(neighborhood_venues.shape)
neighborhood_venues.head()

(358, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0-5,29.70617,-82.388015,China Bowl,29.702655,-82.390303,Chinese Restaurant
1,0-5,29.70617,-82.388015,Cedar River Seafood,29.701723,-82.387995,Seafood Restaurant
2,0-5,29.70617,-82.388015,SUBWAY,29.702775,-82.390566,Sandwich Place
3,0-5,29.70617,-82.388015,Flowers Bakery,29.702798,-82.387151,Bakery
4,0-5,29.70617,-82.388015,Volcanic Sushi + Sake,29.702961,-82.390351,Sushi Restaurant


In [31]:
test = neighborhood_venues.groupby('District').count()
test[test['Venue'] > 1]

,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
0-5,5,5,5,5,5,5
0-7,3,3,3,3,3,3
0-8,2,2,2,2,2,2
10-2,9,9,9,9,9,9
10-3,2,2,2,2,2,2
10-6,5,5,5,5,5,5
10-7,2,2,2,2,2,2
2-0,3,3,3,3,3,3
2-4,4,4,4,4,4,4
